***
<a id='beginning'></a> <!--\label{beginning}-->
* [Outline](../0_Introduction/0_introduction.ipynb)
* [Glossary](../0_Introduction/1_glossary.ipynb)
* [4. The Visibility space](4_0_introduction.ipynb)
    * Previous: [4.3 The Visibility Function](4_3_The_Visibility_Function.ipynb)
    * Next: [4.4.2 UV Coverage: The N-element Interferometer](4_4_2_UV_Coverage_The_N-element_Interferometer.ipynb)

***

In [1]:
#import initpython # containing all "hidden" packages (including CSS reader, and apply CSS style)
from initpython import *
css_styling()

Config loaded


---
# Outline

### 4.4.1 UV Tracks

* 4.4.1.A Observing with a baseline in time
    - Variation of the projected baseline
    - Motion in the (u,v) plane
    - e.g. Earth rotation: the baseline as seen from the source
    

* 4.4.1.B Shape of the tracks
    - dependence of baseline length
    - Latitude of observation
    - General case (tracks are ellipsis)
    - parameters of the ellipsis
    
    
* 4.4.1.C Special cases
    - East-West array
    - Polar interferometer (tracks are circle)
    - Equatorial interferometer (tracks are straight lines)


---

---

---


<p class=conclusion>
  <font size=4> **Important things to remember**</font>
  <br>
  <br>
  
&bull; *The measured spatial frequency depends on the length of the **projected** baseline* <br><br>
&bull; *The measured spatial frequency depends on the length of the **projected** baseline*
</p>

---
* [Go back up](#beginning) <!--\ref{beginning}--> 
* Or continue to next section: [4.4.2 UV Coverage: The N-element Interferometer -->](4_4_2_UV_Coverage_The_N-element_Interferometer.ipynb)
---